# Task 3.1 Construct Dynamic Circuits

## Objective : Classical Feedforward and Control Flow

Classical feedforward allows quantum circuits to make decisions based on measurement results, enabling adaptive quantum algorithms where later operations depend on earlier measurement outcomes. A few use cases have been identified like 

- Efficient quantum state prepreation
- Efficient long range entanglement

More information on classical feed forward and dynamic circuits can be found here
https://quantum.cloud.ibm.com/docs/en/guides/classical-feedforward-and-control-flow

### IF Statement with Classical Conditions

The `if_test` method allows conditional operations based on classical register values. This can be used in measurement-based quantum computing, adaptive algorithms and error correction.

An ```else``` block can be added also to ```with``` statement and it is executed when the ```if``` block is not executed 

In [ ]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
 
# Create quantum and classical registers
qubits = QuantumRegister(2)
clbits = ClassicalRegister(2)
circuit = QuantumCircuit(qubits, clbits)
(q0,q1) = qubits
(c0,c1) = clbits
 
# Step 1: Create superposition and measure
circuit.h(q0)                    # Apply Hadamard: |0⟩ → (|0⟩ + |1⟩)/√2
circuit.measure(q0, c0)          # Measure qubit and store result in classical bit
 
# Step 2: Conditional operation based on measurement result
with circuit.if_test((c0, 1)) as else_:
    circuit.x(q1)               # X gate is applied on q1 if c0 == 1
with else_:
    circuit.h(q1)               # H gate is applied on q1 if c0 == 0

# Step 3: Measure c1 to see the effect of conditional operation
circuit.measure(q1, c1)

 
print("Dynamic Circuit with Classical Conditional:")
circuit.draw("mpl")
 
# - If first measurement gives |1⟩ (c0=1): X gate applied, second measurement gives c1=1
# - If first measurement gives |0⟩ (c0=0): H gate is applied on q1, c1 has 50% propability of being 1 and 50% of being 0

# Output {'00': 25%, '10': 25%, '11': 50%}

The same can be done on the value of a classical register of multiple bits

In [ ]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
 
# Create quantum and classical registers
qubits = QuantumRegister(4)
clbits = ClassicalRegister(4)
circuit = QuantumCircuit(qubits, clbits)
(q0,q1,q2,q3) = qubits
(c0,c1,c2,c3) = clbits
 
# Step 1: Create superposition and measure
circuit.h([q0,q1])                 # Apply Hadamard: |00⟩ → (|00⟩ + |01⟩ + |10⟩ + |11⟩)/2
circuit.measure([q0,q1] ,[c0,c1]) # Measure qubits and store result in classical bits
 
# Step 2: Conditional operation based on measurement result
with circuit.if_test((clbits, 0b0000)):
    circuit.x(q2)               # X gate is applied on q2 if c1c0 == 00
    circuit.x(q3)               # x gate is applied on q3 if c1c0 == 00
with circuit.if_test((clbits, 0b0001)):
    circuit.x(q3)               # X gate is applied on q3 if c1c0 == 01    
with circuit.if_test((clbits, 0b0010)):
    circuit.x(q2)               # X gate is applied on q2 if c1c0 == 10
    
    
# Step 3: Measure c1 to see the effect of conditional operation
circuit.measure([q2,q3], [c2,c3])

 
print("Dynamic Circuit for multiple bits register")
circuit.draw("mpl")
 
# - If first measurement gives |00⟩ : X gate applied on q2 and q3, second measurement gives c3c2=11
# - If first measurement gives |01⟩ : X gate applied on q3, second measurement gives c3c2=10
# - If first measurement gives |10⟩ : X gate applied on q2, second measurement gives c3c2=01
# - If first measurement gives |11⟩ : No gate applied, second measurement gives c3c2=00

### Advanced Classical Expressions

Qiskit supports complex classical expressions using the `expr` module, enabling sophisticated conditional logic based on multiple classical bits and their relationships.

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.classical import expr
 
# Create a surface code inspired circuit for error correction
num_qubits = 8
if num_qubits % 2 or num_qubits < 4:
    raise ValueError("num_qubits must be an even integer ≥ 4")
    
# Select qubits that will be measured for syndrome extraction
meas_qubits = list(range(2, num_qubits, 2))  # qubits to measure and reset
 
# Create registers
qr = QuantumRegister(num_qubits, "qr")
mr = ClassicalRegister(len(meas_qubits), "m")  # Measurement results register
qc = QuantumCircuit(qr, mr)
 
# ===== STEP 1: Create Entangled State =====
# Initialize and create local Bell pairs (entangled states)
qc.reset(qr)                    # Initialize all qubits to |0⟩
qc.h(qr[::2])                   # Apply Hadamard to even-indexed qubits
for ctrl in range(0, num_qubits, 2):
    qc.cx(qr[ctrl], qr[ctrl + 1])  # Create Bell pairs: (|00⟩ + |11⟩)/√2
 
# ===== STEP 2: Create Global Entanglement =====
# Entangle neighboring Bell pairs to create larger entangled state
for ctrl in range(1, num_qubits - 1, 2):
    qc.cx(qr[ctrl], qr[ctrl + 1])
 
# ===== STEP 3: Syndrome Extraction =====
# Measure boundary qubits for error detection (syndrome measurement)
for k, q in enumerate(meas_qubits):
    qc.measure(qr[q], mr[k])    # Measure and store result
    qc.reset(qr[q])             # Reset measured qubit to |0⟩ for reuse
 
# ===== STEP 4: Classical Feedforward Correction =====
# Parity-conditioned X corrections based on syndrome measurements
# Each non-measured qubit gets flipped if the XOR parity of all
# preceding measurement bits is 1 (indicating an error)
for tgt in range(num_qubits):
    if tgt in meas_qubits:      # Skip the qubits that were measured
        continue
        
    # Find all measurement registers whose physical qubit index < current target
    left_bits = [k for k, q in enumerate(meas_qubits) if q < tgt]
    if not left_bits:           # Skip if no preceding measurements
        continue
 
    # Build XOR-parity expression using classical expressions
    # This calculates the cumulative XOR of all relevant measurement results
    parity = expr.lift(mr[left_bits[0]])  # Convert first classical bit to Value object
    for k in left_bits[1:]:
        parity = expr.bit_xor(mr[k], parity)  # Calculate cumulative XOR parity
        
    # Apply conditional X gate based on parity calculation
    # If parity == 1 (odd number of 1s in measurements), apply correction
    with qc.if_test(parity):
        qc.x(qr[tgt])
 
# ===== STEP 5: Re-establish Entanglement =====
# Re-entangle the measured and reset qubits back into the global state
for ctrl in range(1, num_qubits - 1, 2):
    qc.cx(qr[ctrl], qr[ctrl + 1])

print("Advanced Dynamic Circuit with Classical Expressions:")
print(f"Total qubits: {num_qubits}, Measured qubits: {meas_qubits}")
print(f"Classical register size: {len(meas_qubits)} bits")
qc.draw('mpl')

# This circuit demonstrates:
# 1. Complex entanglement patterns
# 2. Syndrome measurement for error detection
# 3. Classical computation (XOR parity) on measurement results
# 4. Conditional quantum operations based on classical computation
# 5. Dynamic error correction based on real-time measurements

### Finding backend that support dynamic circuits

Not all backends support dynamic circuits,  The code below finds all backends that your account can access

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

# update <cloud_api_key>, <instance_CRN> with your API Key and Instance CRN from IBM Quantum
# If your login credentials is already saved, you can initialize the service with empty parameters QiskitRuntimeService()

service = QiskitRuntimeService(token="<cloud_api_key>", instance="<instance_CRN>")
dc_backends = service.backends(dynamic_circuits=True)
print(dc_backends)

### Qiskit Runtime Limitations of Classical Control Flow

Important constraints when using classical conditionals in Qiskit:

* **Bit Limit**: Operands in `if_test` statements must be 32 or fewer bits, if you compare a classical register , it has to be less than 32 bits
* **Broadcasting**: Qiskit Runtime cannot broadcast "transfer data to control logic" more than 60 bits at a time (use barriers to manage different broadcasts)
* **No Nesting**: Nested conditionals (if_test inside another if_test) are not supported
* **No Reset/Measure in Conditionals**: Reset or measurement operations inside conditionals are not supported
* **No Arithmetic**: Arithmetic operations within classical expressions are not supported
* ``` for, while``` and ``` switch``` instructions are not supported 

These limitations are important to consider when designing complex classical feedforward circuits.